In [1]:
import pandas as pd
from datetime import datetime
import networkx as nx
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
def read_data(file_path):
    try:
        data = pd.read_excel(file_path, engine='openpyxl')

        if 'Дата_Рождения' in data.columns:
            data['Дата_Рождения'] = pd.to_datetime(data['Дата_Рождения'], errors='coerce')

            today = datetime.today()
            data['Возраст'] = data['Дата_Рождения'].apply(
                lambda x: (today - x).days // 365 if pd.notnull(x) else None
            )               

        return data
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return None

In [23]:
fenotype_data = read_data('./../data/Датасет на хакатон.xlsx')
fenotype_data.fillna(0, inplace=True)
fenotype_data.head(7)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4
5,6,Самка,Голштинская,2019-08-26,7942,7035,35.590611,5,0.01,0.00,7,88.0,86,5
6,7,Самец,Герефорд,2022-03-04,9472,9004,0.000000,4,0.11,0.00,5,0.0,85,2


In [28]:
def rename_columns(data: pd.DataFrame) -> pd.DataFrame:
    column_mapping = {
        'ID_особи': 'id_individual',
        'Пол': 'sex',
        'Порода': 'breed',
        'Дата_Рождения': 'birth_date',
        'Родитель_папа': 'father_id',
        'Родитель_мама': 'mother_id',
        'Удой л/день': 'milk_yield_day',
        'Упитанность': 'body_condition',
        'Коэффициент инбридинга (F)': 'inbreeding_coefficient',
        'Прирост веса кг/день': 'weight_gain_day',
        'Здоровье (1-10)': 'health_score',
        'Фертильность (%)': 'fertility_percentage',
        'Генетическая ценность (баллы)': 'genetic_value'
    }

    # Корректно выбираем существующие колонки для переименования
    columns_to_rename = {key: value for key, value in column_mapping.items() if key in data.columns}

    # Переименовываем только подходящие колонки
    data.rename(columns=columns_to_rename, inplace=True)

    return data


In [29]:
fenotype_data = rename_columns(fenotype_data)
fenotype_data

,id_individual,sex,breed,birth_date,father_id,mother_id,milk_yield_day,body_condition,inbreeding_coefficient,weight_gain_day,health_score,fertility_percentage,genetic_value,Возраст
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Самка,Симентальская,2022-08-05,9747,5373,0.000000,4,0.03,0.97,9,0.0,89,2
9996,9997,Самец,Айрширская,2023-04-10,3037,3474,0.000000,4,0.03,1.09,9,0.0,87,1
9997,9998,Самка,Швицкая,2023-05-11,1034,8361,25.774777,4,0.01,0.00,6,85.0,93,1
9998,9999,Самка,Айрширская,2019-06-13,8307,70,0.000000,3,0.10,1.39,5,0.0,77,5


In [5]:
genotype_data = read_data('./../data/Генетические мутации хакатон.xlsx')
genotype_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2


In [6]:
merged_data = pd.merge(genotype_data, fenotype_data, on='ID_особи')
merged_data.fillna(0, inplace=True)
merged_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи,Пол,...,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1,Самка,...,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1,Самка,...,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1,Самка,...,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2,Самка,...,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6


In [7]:
def check_mutation(genotype, ref, alt):
    if genotype == f"{alt}/{alt}":
        return 1
    elif genotype == f"{ref}/{ref}":
        return 0
    elif genotype == f"{ref}/{alt}" or genotype == f"{alt}/{ref}":
        return 0.5
    else:
        return -1
    
merged_data['is_mutation'] = merged_data.apply(lambda row: check_mutation(row['Генотип коровы'], row['ref'], row['alt']), axis = 1)
merged_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи,Пол,...,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,is_mutation
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1,Самка,...,150,0.000000,4,0.17,0.00,8,76.0,93,1,1.0
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1,Самка,...,150,0.000000,4,0.17,0.00,8,76.0,93,1,1.0
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1,Самка,...,150,0.000000,4,0.17,0.00,8,76.0,93,1,0.0
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.0
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.5
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.0
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2,Самка,...,6500,37.478242,4,0.15,1.18,5,0.0,87,6,0.5


In [8]:
def calculate_final_trait(row):
    base_value = row[row['Признак']]
    return base_value + row['beta'] * row['is_mutation']

merged_data['Итоговое значение признака'] = merged_data.apply(calculate_final_trait, axis = 1)
merged_data.head(7)

,mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,ID_особи,Пол,...,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,is_mutation,Итоговое значение признака
0,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,1,Самка,...,0.000000,4,0.17,0.00,8,76.0,93,1,1.0,85.520000
1,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,1,Самка,...,0.000000,4,0.17,0.00,8,76.0,93,1,1.0,-6.760000
2,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1,Самка,...,0.000000,4,0.17,0.00,8,76.0,93,1,0.0,8.000000
3,mut_000004,27,25531753,C,G,Коэффициент инбридинга (F),1.04,C/C,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.0,0.150000
4,mut_000005,105,9093860,T,C,Упитанность,0.15,T/C,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.5,4.075000
5,mut_000006,103,23951303,T,G,Удой л/день,-9.32,T/T,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.0,37.478242
6,mut_000007,41,20770253,A,C,Генетическая ценность (баллы),-8.65,A/C,2,Самка,...,37.478242,4,0.15,1.18,5,0.0,87,6,0.5,82.675000


In [9]:
def calculate_penalty(value, optimal_range, weight):
    if isinstance(optimal_range, tuple):
        lower, upper = optimal_range
        if value < lower:
            return (lower - value) * weight
        elif value > upper:
            return (value - upper) * weight
    elif isinstance(optimal_range, float) or isinstance(optimal_range, int):
        return abs(value - optimal_range) * weight
    return 0

In [10]:
def evaluate_cow(cow, common_weights, direction_weights, direction):
    penalty = 0
    penalty += calculate_penalty(cow['Коэффициент инбридинга (F)'], (0, 0.24), common_weights['Коэффициент инбридинга (F)']) 
    penalty += calculate_penalty(cow['Здоровье (1-10)'], (8, 10), common_weights['Здоровье (1-10)']) 
    penalty += calculate_penalty(cow['Упитанность'], (3, 4), common_weights['Упитанность']) 
    penalty += calculate_penalty(cow['Фертильность (%)'], 70, common_weights['Фертильность (%)']) 
    penalty += calculate_penalty(cow['Генетическая ценность (баллы)'], (80, 100), common_weights['Генетическая ценность (баллы)'])

    if direction in direction_weights: 
        dir_weights = direction_weights[direction] 
        penalty += calculate_penalty(cow['Удой л/день'], 30, dir_weights['Удой л/день']) # Укажите свои оптимальные значения 
        penalty += calculate_penalty(cow['Прирост веса кг/день'], 0.7, dir_weights['Прирост веса кг/день']) 
    return penalty

In [11]:
my_cow = {
    'ID_особи': 2,
    'Пол': 'Самец',	
    'Порода': 'Айрширская',
    'Дата_Рождения': '2021-10-19',	
    'Родитель_папа': 3532,
    'Родитель_мама': 6271,
    'Удой л/день': 20,
    'Упитанность': 4,
    'Коэффициент инбридинга (F)': 0.02,	
    'Прирост веса кг/день': 1.22,	
    'Здоровье (1-10)': 5,	
    'Фертильность (%)': 85,	
    'Генетическая ценность (баллы)': 70,
    'Возраст': 3,
}

common_weights = {
    "Коэффициент инбридинга (F)": 2.0,
    "Здоровье (1-10)": 1.5,
    "Упитанность": 1.2,
    "Возраст": 1.0,
    "Фертильность (%)": 1.7,
    "Генетическая ценность (баллы)": 1.4
}

direction_weights = {
    'молочное': {
        'Удой л/день': 2.0,
        'Прирост веса кг/день': 1.0
    },
    'мясное': {
        'Удой л/день': 1.0,
        'Прирост веса кг/день': 2.0
    },
    'комбинированное': {
        'Удой л/день': 1.5,
        'Прирост веса кг/день': 1.5
    },

}


direction = 'молочное'
fenotype_data['Penalty'] = fenotype_data.apply(
    lambda row: evaluate_cow(row, common_weights, direction_weights, direction),
    axis = 1
)

fenotype_data.head(7)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,Penalty
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1,70.900000
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6,138.936484
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1,154.739596
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2,37.049199
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4,182.700000
5,6,Самка,Голштинская,2019-08-26,7942,7035,35.590611,5,0.01,0.00,7,88.0,86,5,45.181223
6,7,Самец,Герефорд,2022-03-04,9472,9004,0.000000,4,0.11,0.00,5,0.0,85,2,184.200000


In [16]:
my_cow_penalty = evaluate_cow(pd.Series(my_cow), common_weights, direction_weights, direction)
my_cow_penalty

64.52

In [13]:
def calculate_compatibility(cow, my_cow, cow_penalty, my_cow_penalty):
    if cow['Пол'] == my_cow['Пол']:
        return 0
    return 100 - (abs(cow_penalty - my_cow_penalty) / max(cow_penalty, my_cow_penalty)) * 100

In [14]:
fenotype_data['Compatibility'] = fenotype_data.apply( 
    lambda row: calculate_compatibility(row, pd.Series(my_cow), row['Penalty'], my_cow_penalty), 
    axis = 1
)
fenotype_data.head(7)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,Penalty,Compatibility
0,1,Самка,Симентальская,2022-12-22,1996,150,0.000000,4,0.17,0.00,8,76.0,93,1,70.900000,91.001410
1,2,Самка,Герефорд,2018-02-07,7154,6500,37.478242,4,0.15,1.18,5,0.0,87,6,138.936484,46.438486
2,3,Самец,Швицкая,2023-07-04,9107,1784,21.730202,3,0.07,0.00,5,0.0,70,1,154.739596,0.000000
3,4,Самец,Герефорд,2022-10-08,3959,1785,22.625400,2,0.12,0.00,9,82.0,84,2,37.049199,0.000000
4,5,Самка,Айрширская,2020-10-07,6921,4409,0.000000,3,0.14,0.00,6,0.0,87,4,182.700000,35.314724
5,6,Самка,Голштинская,2019-08-26,7942,7035,35.590611,5,0.01,0.00,7,88.0,86,5,45.181223,70.026694
6,7,Самец,Герефорд,2022-03-04,9472,9004,0.000000,4,0.11,0.00,5,0.0,85,2,184.200000,0.000000


In [15]:
filtered_data = fenotype_data[(fenotype_data['Порода'] == my_cow['Порода'])]

sorted_data = filtered_data.sort_values(by='Compatibility', ascending=False)
sorted_data.head(10)

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),Возраст,Penalty,Compatibility
9851,9852,Самка,Айрширская,2018-09-16,3345,9729,39.028498,3,0.02,1.20,7,92.0,75,6,64.456996,99.902350
1139,1140,Самка,Айрширская,2019-04-19,7290,7719,37.016320,3,0.07,1.46,7,91.0,71,5,64.592640,99.887542
2162,2163,Самка,Айрширская,2018-11-06,7728,1002,38.477607,4,0.07,1.08,8,91.0,72,6,64.235213,99.558607
4879,4880,Самка,Айрширская,2020-06-17,3984,5225,38.325703,4,0.14,1.24,7,90.0,71,4,65.291405,98.818519
8817,8818,Самка,Айрширская,2021-08-25,5340,6271,38.216044,4,0.18,0.00,8,90.0,71,3,63.732088,98.778809
6930,6931,Самка,Айрширская,2022-03-28,9598,419,22.565812,3,0.06,0.00,7,94.0,76,2,63.468376,98.370080
4984,4985,Самка,Айрширская,2021-08-06,7340,2867,21.555492,3,0.08,0.00,9,95.0,76,3,65.689016,98.220379
3574,3575,Самка,Айрширская,2019-10-18,7080,8110,39.180880,5,0.08,0.00,8,91.0,73,5,65.761760,98.111729
585,586,Самка,Айрширская,2018-04-18,2845,1437,37.615183,2,0.15,1.09,5,88.0,70,6,65.920366,97.875671
4649,4650,Самка,Айрширская,2021-07-25,1195,2222,23.322508,2,0.05,0.00,5,87.0,70,3,62.654983,97.109397
